In [2]:
import numpy as np

np.random.seed(0)
n = 100
# Note that NumPy in default uses 64-bit floating-points or 64-bit integers,
# which is different from 32-bit floating point typically used in deep learning,
# so we explicitly cast the data type
a = np.random.normal(size=n).astype(np.float32)
b = np.random.normal(size=n).astype(np.float32)
c = a + b

def vector_add(a, b, c):
    for i in range(n):
        c[i] = a[i] + b[i]

d = np.empty(shape=n, dtype=np.float32)
vector_add(a, b, d)
np.testing.assert_array_equal(c, d)

In [3]:
def get_abc(shape, constructor=None):
    np.random.seed(0)
    # Note that NumPy in default uses 64-bit floating-points or 64-bit integers,
    # which is different from 32-bit floating point typically used in deep learning,
    # so we explicitly cast the data type
    a = np.random.normal(size=n).astype(np.float32)
    b = np.random.normal(size=n).astype(np.float32)
    c = np.empty_like(a)
    if constructor:
        a, b, c = [constructor(x) for x in (a, b, c)]
    return a, b, c

In [4]:
import tvm
from tvm import te # te stands for tensor expression

In [5]:
def vector_add(n):
    A = te.placeholder((n,), name='a')
    B = te.placeholder((n,), name='b')
    C = te.compute(A.shape, lambda i: A[i] + B[i], name='c')
    return A, B, C

A, B, C = vector_add(n)

In [6]:
print(f"type(A) {type(A)}")
print(f"type(C) {type(C)}")

type(A) <class 'tvm.te.tensor.Tensor'>
type(C) <class 'tvm.te.tensor.Tensor'>


In [7]:
s = te.create_schedule(C.op)
tvm.lower(s, [A, B, C], simple_mode=True)

mod = tvm.build(s, [A, B, C])
print(f"type(mod) {type(mod)}")

x = np.ones(2)
y = tvm.nd.array(x)
print(f"type(y) {type(y)}")
print(f"y.asnumpy() {type(y.asnumpy())}")

a, b, c = get_abc(100, tvm.nd.array)
mod(a, b, c)
np.testing.assert_array_equal(a.asnumpy() + b.asnumpy(), c.asnumpy())

type(mod) <class 'tvm.driver.build_module.OperatorModule'>
type(y) <class 'tvm.runtime.ndarray.NDArray'>
y.asnumpy() <class 'numpy.ndarray'>
